The first thing we have to do is checking if we have support for a GPU.

It's required to enable GPU hardware accelerator over `Edit > Notebook Settings`

In [ ]:
!nvidia-smi

## Next bunch of code is to set up `Stable Diffussion`

1. Install Dependencies

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

2. Enable external Colab widgets

This allows to connect to hugging face, which will be used to download stable difussion models

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

3. Connect to Hugging Face

For this step you will have to create an account for `Hugging Face` if you haven't already; and accept their terms and conditions. The widget have a link to the website where you would be doing that

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

4. Initialize `Stable Diffussion Pipeline` 

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)

5. Move pipeline to GPU

In [ ]:
pipe = pipe.to("cuda")

6. Run!

In [ ]:
from torch import autocast
import io


def dream(prompt: str):
  with autocast("cuda"):
    image = pipe(prompt)["sample"][0]  

  byteIO = io.BytesIO()
  image.save(byteIO, format='PNG')
  return byteIO.getvalue()

----------------------------------

## CREATE THE API!!!

At this stage we have a system that can create images given a written prompt. Let's use **Fast API** to create an API which given a query `prompt="A pug dressed like Napoleon` generates the image and returns it back.

Example:

`http://your-public-address.com?prompt="A pug dressed like Napoleon"`

1. Install dependencies

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

2. Create the API definition

In [ ]:
from fastapi import FastAPI, Response
import asyncio
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/")
async def root(prompt: str):
    return Response(content=dream(prompt), media_type="image/png")

3. Run the API

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)